# **NLP 2일차**
## 과제 2번
### 1. 뉴스 기사에서 특정 문자들만 추출

In [40]:
import re
import requests
from bs4 import BeautifulSoup as bs
url = "https://edition.cnn.com/2023/07/24/football/new-zealand-switzerland-norway-womens-world-cup-2023-spt-intl/index.html"

response = requests.get(url)
r1 = response.content

soup = bs(r1, 'html.parser')
article_origin = soup.find('div',class_='article__content').get_text()

# print(article_origin)
# for elem in article_origin.split("\n"):
#     if len(elem.split()) != 0:
#         for item in elem.split():
#             res.append(item)

pattern = re.compile('[0-9]+-+[0-9]')
print(pattern.findall(article_origin))


['1-0', '0-0', '2-0']


### 2. Wine Mag Data

In [70]:
# Importing Module
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/choejaehun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/choejaehun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [71]:
# 사용 데이터 로드
path = './winemag-data-130k-v2.csv'
df = pd.read_csv(path)

In [72]:
# 새로운 데이터 프레임 생성후 범위조정한 데이터 저장
mydata = df.head(5000)

In [73]:
# 결측치 확인
mydata.isna().sum()

Unnamed: 0                  0
country                     3
description                 0
designation              1477
points                      0
price                     343
province                    3
region_1                  792
region_2                 3044
taster_name              1031
taster_twitter_handle    1206
title                       0
variety                     0
winery                      0
dtype: int64

In [74]:
# 반복문을 이용한 토크나이징 및 전처리
# for i, wine_desc in enumerate(mydata['description']):
#     words = word_tokenize(wine_desc) # 토크나이징
#     clean_word = []
#     for word in words:
#         word = word.lower() # 소문자로 처리
#         if word not in stopwords.words('english'): #불용어 제거
#             clean_word.append(word)
    
#     mydata[i] = ' '.join(clean_word) # 전처리된 문장을 저장

In [75]:
# CountVectorizer 객체 생성후 빈도수 3이하 제거
v = CountVectorizer(min_df=3)
v.fit_transform(mydata['description']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [76]:
# TfidfVectorizer 객체 생성후 영어 불용어사전 참조후 제거
tfidf = TfidfVectorizer(stop_words='english')

In [77]:
tfidf_m = tfidf.fit_transform(mydata['description'])
tfidf_m.toarray()
print(f"tfidf_m.shape : {tfidf_m.shape}")

tfidf_m.shape : (5000, 8179)


In [78]:
# 코사인 유사도 계산
cos_sim = cosine_similarity(tfidf_m, tfidf_m)
print(cos_sim.shape)

(5000, 5000)


In [79]:
data = dict(zip(mydata['title'], mydata.index))
idx = data['Nicosia 2013 Vulkà Bianco  (Etna)']

In [80]:
def recommend(title, cosine_sim=cos_sim):
    idx = data[title]
    tmp = cosine_sim[idx]
    sim_scores = list(enumerate(cosine_sim[idx]))
    ss = sorted(sim_scores, key=lambda x:-x[1])[1:11]

    res = [idx[0] for idx in ss]
    # print(f"res : {res}")
    # print(f"ss : {ss}")
    return df['title'].iloc[res]


In [81]:
recommend('Simonnet-Febvre 2015  Chablis')

49       Vignerons de Bel Air 2011 Eté Indien  (Brouilly)
3353    Jean-Marie Challand 2012 Les Tilles  (Mâcon-Vi...
3246                 Liebart-Régnier NV Brut  (Champagne)
3567                       Domaine de Rome 2015  Sancerre
2658          Château Tour de Lagarde 2009  Saint-Émilion
4091     Casca Wines 2013 Santos da Casa Red (Alentejano)
30      Domaine de la Madone 2012 Nouveau  (Beaujolais...
4142    Simonnet-Febvre 2015 Côte de Lechet Premier Cr...
4563    Les Vignerons Réunis de Monségur 2014 Château ...
1558                        Château Margaux 2009  Margaux
Name: title, dtype: object